# Análise de Desempenho dos Estudantes com K-Means

Este notebook realiza uma análise de agrupamento (clustering) dos estudantes usando o algoritmo K-Means, seguindo o padrão do projeto KNN, para fins de comparação e estudo de técnicas não supervisionadas.

# 1. Importação das Bibliotecas

Utilizaremos pandas, numpy, matplotlib, seaborn e scikit-learn (KMeans, métricas de cluster).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.preprocessing import StandardScaler, LabelEncoder

# 2. Carregamento do Dataset

O dataset utilizado é o mesmo do projeto KNN, contendo informações sobre desempenho de estudantes em exames.

In [ ]:
import kagglehub

# Baixar o dataset do Kaggle
path = kagglehub.dataset_download("spscientist/students-performance-in-exams")
print("Path to dataset files:", path)

# Carregar o arquivo CSV
csv_path = path + "/StudentsPerformance.csv"
df = pd.read_csv(csv_path)
df.head()

# 3. Análise Exploratória dos Dados

Verificação do formato, tipos de dados, valores nulos e estatísticas descritivas.

In [ ]:
print('Formato do dataset:', df.shape)
df.info()
print('\nValores nulos por coluna:')
print(df.isnull().sum())
print('\nEstatísticas das notas:')
print(df[['math score', 'reading score', 'writing score']].describe())

# 4. Visualização das Notas

Histogramas e boxplots para visualizar a distribuição das notas dos estudantes.

In [ ]:
import os
os.makedirs('imagens', exist_ok=True)

# Histogramas das notas
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for idx, col in enumerate(['math score', 'reading score', 'writing score']):
    sns.histplot(df[col], bins=20, ax=axes[idx], kde=True)
    axes[idx].set_title(f'Distribuição: {col}')
plt.tight_layout()
plt.savefig('imagens/histograma_notas.png')
plt.show()
plt.close(fig)

# Boxplot das notas
fig2, ax2 = plt.subplots(figsize=(10, 6))
sns.boxplot(data=df[['math score', 'reading score', 'writing score']], ax=ax2)
ax2.set_title('Boxplot das Notas')
plt.tight_layout()
plt.savefig('imagens/boxplot_notas.png')
plt.show()
plt.close(fig2)

# 5. Estatísticas das Notas e Interpretação

Análise estatística das notas dos estudantes, incluindo média, desvio padrão, valores mínimos e máximos.

In [ ]:
# Estatísticas já exibidas acima. Podemos complementar com visualização de pairplot para explorar relações.
if 'passed' not in df.columns:
    df['passed'] = (df['math score'] >= 60).astype(int)
sns.pairplot(df, hue='passed')
plt.savefig('imagens/pairplot_passed.png')
plt.show()

# 6. Pré-processamento dos Dados

Codificação de variáveis categóricas e normalização das variáveis numéricas para o K-means.

In [ ]:
# Codificação das variáveis categóricas
df_encoded = df.copy()
categorical_cols = df_encoded.select_dtypes(include=['object']).columns
for col in categorical_cols:
    df_encoded[col] = LabelEncoder().fit_transform(df_encoded[col])

# Normalização das variáveis numéricas
scaler = StandardScaler()
features = ['math score', 'reading score', 'writing score']
df_encoded[features] = scaler.fit_transform(df_encoded[features])
df_encoded.head()

# 7. Treinamento do Modelo K-Means

Aplicação do algoritmo K-means para agrupar os estudantes. O número de clusters será definido com base na análise exploratória e/ou métodos como o cotovelo (elbow method).

In [ ]:
# Encontrando o número ideal de clusters (Elbow Method)
wcss = []
X = df_encoded[features]
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X)
    wcss.append(kmeans.inertia_)
plt.figure(figsize=(8, 5))
plt.plot(range(1, 11), wcss, marker='o')
plt.xlabel('Número de clusters (K)')
plt.ylabel('WCSS')
plt.title('Método do Cotovelo')
plt.savefig('imagens/elbow_kmeans.png')
plt.show()

In [ ]:
# Treinando o modelo K-means com o número de clusters escolhido (exemplo: 2)
k = 2
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)
df_encoded['cluster'] = clusters
print('Centroides:', kmeans.cluster_centers_)

# 8. Avaliação do Modelo K-Means

Avaliação dos clusters formados usando métricas como silhouette score e visualização dos agrupamentos.

In [ ]:
# Silhouette Score
sil_score = silhouette_score(X, clusters)
print(f'Silhouette Score: {sil_score:.3f}')

# Visualização dos clusters (usando duas notas para plot 2D)
plt.figure(figsize=(8,6))
sns.scatterplot(x='math score', y='reading score', hue='cluster', data=df_encoded, palette='Set1')
plt.title('Clusters formados pelo K-means')
plt.savefig('imagens/clusters_kmeans.png')
plt.show()

# 9. Relatório Final

Resumo dos resultados obtidos, análise dos clusters, possíveis melhorias e comparação com métodos supervisionados (KNN).